# Identify tags in airline database

## Pending to complete the data creation

# Alternative solution

## Architecture
    - Two inputs: 
        previous words seq_inputf = [W_i-k, ..., W_i-1, W_i] 
        Next words seq_inputb = [W_i, W_i+1, ...,W_i+k]
    - Two embeding layers with shared weights
    - Two LSTM, one forward and one backward
    - A final concatenation and a dense layer

In [18]:
# build the model: Simple LSTM with embedings
print('Build model 1')
seq_input_f = tf.keras.layers.Input(shape=([input_seq_length]), name='prev1') 
seq_input_b = tf.keras.layers.Input(shape=([input_seq_length]), name='prev2') 
    
embeds = tf.keras.layers.Embedding(input_vocabulary_size, embedding_size)

embeds1 = embeds(seq_input_f)
embeds2 = embeds(seq_input_b)

lstm_f = tf.keras.layers.LSTM(128)(embeds_f)
lstm_b = tf.keras.layers.LSTM(128, go_backwards=True)(embeds_b)

concat = tf.keras.layers.concatenate([lstm_f, lstm_b], axis=-1)

    
output = tf.keras.layers.Dense(output_length, activation='softmax')(concat)


model1 = tf.keras.models.Model(inputs=[seq_input_f, seq_input_b], outputs=output)
model1.summary()

# Optimizer
adam_optimizer = tf.keras.optimizers.Adam()
model1.compile(loss='sparse_categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])


Build model 1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
prev1 (InputLayer)              (None, 10)           0                                            
__________________________________________________________________________________________________
prev2 (InputLayer)              (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 10, 64)       36672       prev1[0][0]                      
                                                                 prev2[0][0]                      
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 128)          98816       embedding_5[0][0]             

In [ ]:
#Fit model

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR)

history = model1.fit([X_trn_f, X_trn_b], y_trn, batch_size=128, epochs=20,
           validation_data=([X_tst_f, X_trs_b], y_tst), callbacks=[tb_callback])
